In [66]:
import sys
import pandas as pd
import numpy as np

# INPUT FILE
scen_file = 'esg_시나리오.xlsx'
in_file = '지수산출기초데이터.xlsx'
out_file_1 = 'convert_by_isu_'+ in_file   # xlsx로만 저장되네...
out_file_2 = 'convert_by_val_'+ in_file   # xlsx로만 저장되네...


df = pd.read_excel(scen_file,dtype='unicode')
df = df['정기개편일']
df.drop_duplicates(inplace=True)
phase = df.values.tolist()

# input file 을 읽는다
#df = pd.read_csv(in_file, sep=',',dtype='unicode', parse_dates=True)
df = pd.read_excel(in_file, sep=',',dtype='unicode')

#숫자로 봐야하는 컬럼의 dtype을 변경한다
cols=df.columns.drop(['DATE','CODE'])
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# 필요없는 컬럼 지우기
#df.drop([df.columns[1],df.columns[2],df.columns[4],df.columns[7],df.columns[10],df.columns[11]],axis=1,inplace=True)

#
df.set_index('DATE', inplace=True)
df.sort_index(axis = 0, level=0, inplace=True, ascending=True)

# START - 1. 종목별 세부 정보 만들기. MULTI INDEX level 0 => 종목, level 1 => 세부정보
# 각 개편일별로 끊어서 list에 추가함
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # TABLE RESHAPE - 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['CODE'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    # - 종목코드를 열 index level 0로
    df2.columns = df2.columns.swaplevel(0,1)
    # - 열 index level 1(종목별 항목들)을 sorting
    df2.sort_index(axis = 1, level=0, inplace=True)
    df2.apply(pd.to_numeric, errors='ignore')
    df_list.append(df2)

# 엑셀파일저장
with pd.ExcelWriter(out_file_1) as writer:
    for item in df_list :
        item.to_excel(writer, sheet_name=item.index[0])

print ('** FILE CREATED : ' + out_file_1)
# END - 1. 종목별 세부 정보 만들기. MULTI INDEX level 0 => 종목, level 1 => 세부정보

# START - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목
# 각 개편일별로 끊어서 list에 추가함
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # TABLE RESHAPE - 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['CODE'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    df2['BACE PRICE'].apply(pd.to_numeric, errors='ignore')
    df_list.append(df2)
# 엑셀파일저장
with pd.ExcelWriter(out_file_2) as writer:
    for item in df_list :
        item.to_excel(writer, sheet_name=item.index[0])
        
print ('** FILE CREATED : ' + out_file_2)
# END - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목

#완료
print ('converting 작업을 완료하였습니다.')
# 종목코드 열로 옮기는 작업
#  - 키 추가
#df2.set_index(['ISU_SRT_CD'], append=True, inplace=True) #index 추가시 append param 사용함
# - 열로 옮김
#df2 = df2.unstack(level=1)
# - 종목코드를 열 index level 0로
#df2.columns = df2.columns.swaplevel(0,1)
# - 열 index level 1(종목별 항목들)을 sorting
#df2.sort_index(axis = 1, level=0, inplace=True)

# index 세팅
#df.set_index(['TRADE_DATE','ISU_SRT_CD'], inplace=True)

# isu_srt_cd (level = 1)을 열로 보내기
#df2 = df.unstack(level=1)

# isu_srt_cd 를 column index level 0로 바꾸기
# 종목의 세부 정보는 level 1로
#df2.columns = df2.columns.swaplevel(0,1)

# 소팅하기
#df2.sort_index(axis = 1, level=0, inplace=True)

# 결과를 엑셀로 보내기
#df2.to_excel(out_file,sheet_name='test1')


** FILE CREATED : convert_by_isu_지수산출기초데이터.xlsx
** FILE CREATED : convert_by_val_지수산출기초데이터.xlsx
converting 작업을 완료하였습니다.


In [11]:
my_df = df_list[0]
my_df.columns.levels[0]

Index(['BAS_PRICE', 'CLOSE_PRICE', 'IDX_LISTED_STOCKS', 'COMP_RT', 'STYLE_RT'], dtype='object')

In [ ]:
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['ISU_SRT_CD'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    # - 종목코드를 열 index level 0로
    df2.columns = df2.columns.swaplevel(0,1)
    # - 열 index level 1(종목별 항목들)을 sorting
    df2.sort_index(axis = 1, level=0, inplace=True)
    df_list.append(df2)
        


In [ ]:
df_list[1].index[0]

In [ ]:
scen_file = 'esg_시나리오.xlsx'
df3 = pd.read_excel(scen_file,dtype='unicode')

In [ ]:
#df3.set_index('정기개편일', inplace=True)
df3 = df3['정기개편일']

In [ ]:
df3.drop_duplicates(inplace=True)

In [ ]:
price = mydf['CLOSE_PRICE'].astype('int64')

In [ ]:
mydf = df_list[0]["20161004" : "20161230"]
mydf

In [ ]:
shrs = mydf['IDX_LISTED_STOCKS'].astype('int64')

In [ ]:
iif = mydf['STYLE_RT'].astype(float)

In [ ]:
portion = price*shrs*iif

In [ ]:
sum = portion.sum(axis=1,skipna = True)

In [ ]:
portion.to_excel('portion.xlsx')

In [ ]:
cap = price.mul(shrs)

In [ ]:
portion = cap.mul(iif)

In [ ]:
portion.to_excel('portion.xlsx')